## Setup — Import necessary libraries
This notebook uses NumPy, pandas, Matplotlib, Seaborn, scikit-image and SciPy to load, preprocess, segment, and analyze microscopy images

Quick links:
- NumPy: https://numpy.org/doc/
- pandas: https://pandas.pydata.org/docs/
- Matplotlib: https://matplotlib.org/stable/contents.html
- Seaborn: https://seaborn.pydata.org/
- scikit-image: https://scikit-image.org/docs/stable/
- SciPy ndimage: https://docs.scipy.org/doc/scipy/reference/ndimage.html


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import skimage as ski
from scipy import ndimage as ndi

In [ ]:
# !pip install pooch
# !pip install seaborn

### Load in data from `skimage`

In [ ]:
image = ski.data.human_mitosis()

In [ ]:
image.shape, image.dtype

In [ ]:
plt.figure(figsize=(12,5))
plt.imshow(image,cmap='gray');

### Preprocessing — smoothing and contrast enhancement
Apply a Gaussian filter to reduce high-frequency noise, then use CLAHE (adaptive histogram equalization) to improve local contrast before thresholding
- filters.gaussian: https://scikit-image.org/docs/stable/api/skimage.filters.html
- exposure.equalize_adapthist: https://scikit-image.org/docs/stable/api/skimage.exposure.html

In [ ]:
# denoise & enhance contrast
img_smooth = ski.filters.gaussian(image, sigma=1.0)
img_eq = ski.exposure.equalize_adapthist(img_smooth, clip_limit=0.009)

In [ ]:
plt.figure(figsize=(12,5))
plt.imshow(img_eq, cmap='gray');

### Segmentation — Otsu thresholding
Compute a global threshold using Otsu's method and create a binary mask.

Otsu: https://scikit-image.org/docs/stable/api/skimage.filters.html#skimage.filters.threshold_otsu


In [ ]:
fig, ax = ski.filters.try_all_threshold(img_eq, figsize=(10,15), verbose=False)
plt.show()

In [ ]:
# Threshold_Otsu
th = ski.filters.threshold_otsu(img_eq)
binary = img_eq > th

fig, ax = plt.subplots(ncols=2, figsize=(15,10))
ax[0].imshow(image,cmap='gray')
ax[0].set_title('Original')
ax[0].set_axis_off()
ax[1].imshow(binary)
ax[1].set_title('Otsu thresholding)')
ax[1].set_axis_off()
plt.show()

In [ ]:
th

### Distance transform — prepare for marker-based watershed

Smooth the binary mask to remove small irregularities and compute the Euclidean distance transform. This highlights object centers and aids marker generation for watershed.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.distance_transform_edt.html

In [ ]:
# Adding Guassian blur before calcuating distance transform 
smth_binary = ndi.gaussian_filter(binary.astype(float), sigma=2)
distance = ndi.distance_transform_edt(smth_binary > 0.5)

In [ ]:
plt.imshow(distance);

### Detect local maxima (marker detection)
Find local peaks in the distance map to create markers (approximate object centers).

peak_local_max docs: https://scikit-image.org/docs/stable/api/skimage.feature.html

In [ ]:
# Detect local maxima (cell centers)
max_coords = ski.feature.peak_local_max(distance, labels=binary, min_distance=1, footprint=np.ones((10,10)))

In [ ]:
markers = np.zeros_like(binary, dtype=int)
for i, coord in enumerate(max_coords, start=1):
    markers[tuple(coord)] = i

### Watershed segmentation
Apply marker-controlled watershed on the inverted distance map to separate touching objects. Use the binary mask to limit the segmentation region.

watershed: https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimage.segmentation.watershed

In [ ]:
# Apply watershed on the inverted distance
labels_ws = ski.segmentation.watershed(-distance, markers, mask=binary)
plt.imshow(labels_ws);

In [ ]:
print(distance.shape, binary.shape, markers.shape)


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 10))
ax[0].imshow(binary, cmap='gray'); ax[0].set_title('Binary Mask')
ax[1].imshow(distance, cmap='magma'); ax[1].set_title('Distance Transform')
ax[2].imshow(labels_ws, cmap='cividis'); ax[2].set_title('Separated Cells')
for ax in ax: ax.axis('off')
plt.show()


### Getting some metrics

`regionprops_table`: https://scikit-image.org/docs/stable/api/skimage.measure.html#skimage.measure.regionprops_table

In [ ]:
props = ski.measure.regionprops_table(labels_ws, properties=('label','area_filled','perimeter','centroid','equivalent_diameter','eccentricity',
                                                 'solidity','extent','major_axis_length','minor_axis_length'))
df = pd.DataFrame(props)
df.head()

In [ ]:
num_cells = len(df)
print("Number of segmented cells:", num_cells)

In [ ]:
df['smoothness'] = df['area_filled'] / df['perimeter'].replace(0, pd.NA)
df['circularity'] = 4 * np.pi * df['area_filled'] / (df['perimeter'] ** 2)
df['roundness'] = (4 * df['area_filled']) / (np.pi * (df['major_axis_length'] ** 2)) 

In [ ]:
thres = df.query("equivalent_diameter>10")

In [ ]:
sns.histplot(data=thres, x='equivalent_diameter', kde=True)

### Visualizations and exploration

Plot histograms, boxplots and a correlation heatmap to inspect distributions and relationships among measured features.

In [ ]:
sns.histplot(data=df, x='area_filled', kde=True)

In [ ]:
sns.histplot(data=df, x='equivalent_diameter', kde=True)

In [ ]:
df.columns

In [ ]:
# Select property columns
cols = ['area_filled', 'perimeter', 'equivalent_diameter', 'eccentricity', 
        'solidity', 'extent', 'smoothness', 'circularity',
       'roundness']

In [ ]:
corr = df[cols].corr()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, square=True, fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix of Region Properties", fontsize=14)
plt.tight_layout()
plt.show()
